In [1]:
import os 
import sys
while os.path.split(os.getcwd())[1] != 'RecSysChallenge2023-Team':
    os.chdir('..')
sys.path.insert(1, os.getcwd())

import numpy as np 

In [2]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout

import Daniele.Utils.MyDataManager as dm 
import Daniele.Utils.MatrixManipulation as mm

URMv = dm.getURMviews()
URMo = dm.getURMopen()
ICMt=dm.getICMt()
ICMl=dm.getICMl()


URMv_train, URM_test = split_train_in_two_percentage_global_sample(URMv, train_percentage = 0.80)
ICM_genres =  mm.augmentedICM(ICMt,ICMl)

URM_train = mm.defaultExplicitURM(urmv=URMv_train,urmo=URMo,icml=ICMl,icmt=ICMt, normalize=True, add_aug=True,appendICM=True)

/Users/daniele/Desktop/RecSys/RecSysChallenge2023-Team


100%|██████████| 663370/663370 [00:00<00:00, 1801567.74it/s]


In [3]:
import numpy as np 
import scipy.sparse as sps

profile_length = np.ediff1d(sps.csr_matrix(URM_train).indptr)
profile_length, profile_length.shape

(array([  64,   18,   97, ..., 2655, 2450,  461], dtype=int32), (70352,))

Let's select a few groups of 5% of the users with the least number of interactions.

In [4]:
block_size = int(len(profile_length)*0.05)
block_size

3517

In [5]:
sorted_users = np.argsort(profile_length)
sorted_users

array([70345, 70344, 40646, ..., 70346, 70339, 70347])

In [6]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender

MAP_recommender_per_group = {}

collaborative_recommender_class = {"TopPop": TopPop,
                                   "UserKNNCF": UserKNNCFRecommender,
                                   "ItemKNNCF": ItemKNNCFRecommender,
                                   "P3alpha": P3alphaRecommender,
                                   "RP3beta": RP3betaRecommender,
                                   "PureSVD": PureSVDRecommender,
                                   "NMF": NMFRecommender,
                                   "FunkSVD": MatrixFactorization_FunkSVD_Cython,
                                   "SLIMBPR": SLIM_BPR_Cython,
                                   }

content_recommender_class = {"ItemKNNCBF": ItemKNNCBFRecommender,
                             "ItemKNNCFCBF": ItemKNN_CFCBF_Hybrid_Recommender
                            }

recommender_object_dict = {}

for label, recommender_class in collaborative_recommender_class.items():
    recommender_object = recommender_class(URM_train)
    recommender_object.fit()
    recommender_object_dict[label] = recommender_object

for label, recommender_class in content_recommender_class.items():
    recommender_object = recommender_class(URM_train, ICM_genres)
    recommender_object.fit()
    recommender_object_dict[label] = recommender_object

TopPopRecommender: URM Detected 2 ( 0.0%) users with no interactions.
UserKNNCFRecommender: URM Detected 2 ( 0.0%) users with no interactions.
Similarity column 70352 (100.0%), 1667.55 column/sec. Elapsed time 42.19 sec
ItemKNNCFRecommender: URM Detected 2 ( 0.0%) users with no interactions.
Similarity column 27968 (100.0%), 2852.87 column/sec. Elapsed time 9.80 sec
P3alphaRecommender: URM Detected 2 ( 0.0%) users with no interactions.
P3alphaRecommender: Similarity column 27968 (100.0%), 1831.86 column/sec. Elapsed time 15.27 sec
RP3betaRecommender: URM Detected 2 ( 0.0%) users with no interactions.
RP3betaRecommender: Similarity column 27968 (100.0%), 1776.51 column/sec. Elapsed time 15.74 sec
PureSVDRecommender: URM Detected 2 ( 0.0%) users with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 3.14 sec
NMFRecommender: URM Detected 2 ( 0.0%) users with no interactions.
NMFRecommender: Computing NMF decompos

/bin/sh: free: command not found


Processed 70352 (100.0%) in 0.93 sec. BPR loss is 1.15E-06. Sample per second: 75546
SLIM_BPR_Recommender: Epoch 1 of 300. Elapsed time 0.37 sec
Processed 70352 (100.0%) in 1.20 sec. BPR loss is 3.66E-06. Sample per second: 58572
SLIM_BPR_Recommender: Epoch 2 of 300. Elapsed time 0.64 sec
Processed 70352 (100.0%) in 0.42 sec. BPR loss is 6.15E-06. Sample per second: 165635
SLIM_BPR_Recommender: Epoch 3 of 300. Elapsed time 0.86 sec
Processed 70352 (100.0%) in 0.58 sec. BPR loss is 8.85E-06. Sample per second: 121872
SLIM_BPR_Recommender: Epoch 4 of 300. Elapsed time 1.01 sec
Processed 70352 (100.0%) in 0.73 sec. BPR loss is 1.14E-05. Sample per second: 96363
SLIM_BPR_Recommender: Epoch 5 of 300. Elapsed time 1.17 sec
Processed 70352 (100.0%) in 0.91 sec. BPR loss is 1.44E-05. Sample per second: 77403
SLIM_BPR_Recommender: Epoch 6 of 300. Elapsed time 1.35 sec
Processed 70352 (100.0%) in 1.08 sec. BPR loss is 1.68E-05. Sample per second: 65128
SLIM_BPR_Recommender: Epoch 7 of 300. Elaps

In [7]:
cutoff = 10

for group_id in range(0, 20):
    
    start_pos = group_id*block_size
    end_pos = min((group_id+1)*block_size, len(profile_length))
    
    users_in_group = sorted_users[start_pos:end_pos]
    
    users_in_group_p_len = profile_length[users_in_group]
    
    print("Group {}, #users in group {}, average p.len {:.2f}, median {}, min {}, max {}".format(
        group_id, 
        users_in_group.shape[0],
        users_in_group_p_len.mean(),
        np.median(users_in_group_p_len),
        users_in_group_p_len.min(),
        users_in_group_p_len.max()))
    
    
    users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)
    users_not_in_group = sorted_users[users_not_in_group_flag]
    
    evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_not_in_group)
    
    for label, recommender in recommender_object_dict.items():
        result_df, _ = evaluator_test.evaluateRecommender(recommender)
        if label in MAP_recommender_per_group:
            MAP_recommender_per_group[label].append(result_df.loc[cutoff]["MAP"])
        else:
            MAP_recommender_per_group[label] = [result_df.loc[cutoff]["MAP"]]

Group 0, #users in group 3517, average p.len 10.94, median 11.0, min 0, max 13
EvaluatorHoldout: Ignoring 1584 ( 3.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 66835 Users
EvaluatorHoldout: Processed 1520 (100.0%) in 1.25 sec. Users per second: 1221


IndexError: index 63419 is out of bounds for axis 0 with size 41629

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  

_ = plt.figure(figsize=(16, 9))
for label, recommender in recommender_object_dict.items():
    results = MAP_recommender_per_group[label]
    plt.scatter(x=np.arange(0,len(results)), y=results, label=label)
plt.ylabel('MAP')
plt.xlabel('User Group')
plt.legend()
plt.show()